In [1]:
import transformers

To use data.metrics please install scikit-learn. See https://scikit-learn.org/stable/index.html


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
transformers.BERT_PRETRAINED_MODEL_ARCHIVE_MAP["bert-base-finnish-cased"]="http://dl.turkunlp.org/finbert/torch-transformers/bert-base-finnish-cased/pytorch_model.bin"
transformers.BERT_PRETRAINED_CONFIG_ARCHIVE_MAP["bert-base-finnish-cased"]="http://dl.turkunlp.org/finbert/torch-transformers/bert-base-finnish-cased/config.json"
transformers.tokenization_bert.PRETRAINED_VOCAB_FILES_MAP["vocab_file"]["bert-base-finnish-cased"]="http://dl.turkunlp.org/finbert/torch-transformers/bert-base-finnish-cased/vocab.txt"
transformers.tokenization_bert.PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES["bert-base-finnish-cased"]=512
transformers.tokenization_bert.PRETRAINED_INIT_CONFIGURATION["bert-base-finnish-cased"]={'do_lower_case': False}


In [4]:
model=transformers.BertForMaskedLM.from_pretrained("bert-base-finnish-cased")

In [5]:
model=model.cuda()

In [6]:
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-finnish-cased")

In [7]:
CLS,SEP,MASK=tokenizer.convert_tokens_to_ids(["[CLS]","[SEP]","[MASK]"])
texts=["Minulla on koira.","Se koira on ruskea.","Maistui myös oikein pahalta."]
tokenized_texts=[tokenizer.tokenize(t) for t in texts]
tokenized_texts

[['Minulla', 'on', 'koira', '.'],
 ['Se', 'koira', 'on', 'ruskea', '.'],
 ['Mai', '##st', '##ui', 'myös', 'oikein', 'pahalta', '.']]

In [8]:
texts_ids=[torch.tensor([CLS]+tokenizer.convert_tokens_to_ids(tt)+[SEP]) for tt in tokenized_texts]
texts_ids=torch.nn.utils.rnn.pad_sequence(texts_ids,batch_first=True,padding_value=MASK)
texts_ids=texts_ids.cuda()
resp=model(texts_ids)

In [9]:
print(texts_ids.shape)
print(resp[0].shape)

torch.Size([3, 9])
torch.Size([3, 9, 50105])
